# Домашнее задание HW 4

## Задание HW 4

Домашнее задание состоит из нескольких блоков.


### Эксперименты в ipynb ноутбуках (11 баллов)
- Необходимо будет перебрать $N$ моделей $(N \geq 2)$ матричной факторизации и перебрать у них $K$ гиперпараметров $(K \geq 2)$ **(5 баллов)**
    - Для перебора гиперпараметров можно использовать [`Optuna`](https://github.com/optuna/optuna), [`Hyperopt`](https://github.com/hyperopt/hyperopt)
- Воспользоваться методом приближенного поиска соседей для выдачи рекомендаций. **(2 балла)**
    - Можно использовать любые удобные: [`Annoy`](https://github.com/spotify/annoy), [`nmslib`](https://github.com/nmslib/nmslib) и.т.д
- Добавить 3 "аватаров" (искусственных пользователей) и посмотреть рекомендации итоговой модели на них. Объяснить почему добавили именно таких пользователей. **(2 балла)**
- Придумать как можно обработать рекомендации для холодных пользователей. **(2 балла)**

Примечание: за невоспроизводимый код в ноутбуках (например, нарушен порядок выполнения ячеек, вызываются переменные, которые нигде не были объявлены ранее и.т.п) будут штрафы на усмотрение проверяющего.


### Реализация итоговой модели в сервисе (9 баллов)
- Пробитие бейзлайна $MAP@10 \geq 0.074921$ **(6 баллов)**
- Код сервиса соответствует критериям читаемости и воспроизводимости **(3 балла)**

## Imports

In [62]:
import warnings

warnings.filterwarnings('ignore')

import pandas as pd
from collections import Counter
import time
import numpy as np
from pprint import pprint
import dill
from implicit.als import AlternatingLeastSquares

from rectools.metrics import Precision, Recall, MAP, calc_metrics
from rectools.models import PopularModel, RandomModel, ImplicitALSWrapperModel
from rectools import Columns
from rectools.dataset import Dataset
from rectools.models import ImplicitALSWrapperModel, LightFMWrapperModel
import nmslib

import optuna

from pathlib import Path

from lightfm import LightFM

## Подгружаем данные

In [3]:
DATA_PATH = Path("data_original")

In [4]:
users = pd.read_csv(DATA_PATH / 'users.csv')
items = pd.read_csv(DATA_PATH / 'items.csv')
interactions = pd.read_csv(DATA_PATH / 'interactions.csv')

In [5]:
users

,user_id,age,income,sex,kids_flg
0,973171,age_25_34,income_60_90,М,1
1,962099,age_18_24,income_20_40,М,0
2,1047345,age_45_54,income_40_60,Ж,0
3,721985,age_45_54,income_20_40,Ж,0
4,704055,age_35_44,income_60_90,Ж,0
...,...,...,...,...,...
840192,339025,age_65_inf,income_0_20,Ж,0
840193,983617,age_18_24,income_20_40,Ж,1
840194,251008,NaN,NaN,NaN,0
840195,590706,NaN,NaN,Ж,0


In [6]:
items

,item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords
0,10711,film,Поговори с ней,Hable con ella,2002.0,"драмы, зарубежные, детективы, мелодрамы",Испания,NaN,16.0,NaN,Педро Альмодовар,"Адольфо Фернандес, Ана Фернандес, Дарио Гранди...",Мелодрама легендарного Педро Альмодовара «Пого...,"Поговори, ней, 2002, Испания, друзья, любовь, ..."
1,2508,film,Голые перцы,Search Party,2014.0,"зарубежные, приключения, комедии",США,NaN,16.0,NaN,Скот Армстронг,"Адам Палли, Брайан Хаски, Дж.Б. Смув, Джейсон ...",Уморительная современная комедия на популярную...,"Голые, перцы, 2014, США, друзья, свадьбы, прео..."
2,10716,film,Тактическая сила,Tactical Force,2011.0,"криминал, зарубежные, триллеры, боевики, комедии",Канада,NaN,16.0,NaN,Адам П. Калтраро,"Адриан Холмс, Даррен Шалави, Джерри Вассерман,...",Профессиональный рестлер Стив Остин («Все или ...,"Тактическая, сила, 2011, Канада, бандиты, ганг..."
3,7868,film,45 лет,45 Years,2015.0,"драмы, зарубежные, мелодрамы",Великобритания,NaN,16.0,NaN,Эндрю Хэй,"Александра Риддлстон-Барретт, Джеральдин Джейм...","Шарлотта Рэмплинг, Том Кортни, Джеральдин Джей...","45, лет, 2015, Великобритания, брак, жизнь, лю..."
4,16268,film,Все решает мгновение,NaN,1978.0,"драмы, спорт, советские, мелодрамы",СССР,NaN,12.0,Ленфильм,Виктор Садовский,"Александр Абдулов, Александр Демьяненко, Алекс...",Расчетливая чаровница из советского кинохита «...,"Все, решает, мгновение, 1978, СССР, сильные, ж..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15958,6443,series,Полярный круг,Arctic Circle,2018.0,"драмы, триллеры, криминал","Финляндия, Германия",NaN,16.0,NaN,Ханну Салонен,"Иина Куустонен, Максимилиан Брюкнер, Пихла Вии...","Во время погони за браконьерами по лесу, сотру...","убийство, вирус, расследование преступления, н..."
15959,2367,series,Надежда,NaN,2020.0,"драмы, боевики",Россия,0.0,18.0,NaN,Елена Хазанова,"Виктория Исакова, Александр Кузьмин, Алексей М...",Оригинальный киносериал от создателей «Бывших»...,"Надежда, 2020, Россия"
15960,10632,series,Сговор,Hassel,2017.0,"драмы, триллеры, криминал",Россия,0.0,18.0,NaN,"Эшреф Рейбрук, Амир Камдин, Эрик Эгер","Ола Рапас, Алиетт Офейм, Уильма Лиден, Шанти Р...",Криминальная драма по мотивам романов о шведск...,"Сговор, 2017, Россия"
15961,4538,series,Среди камней,Darklands,2019.0,"драмы, спорт, криминал",Россия,0.0,18.0,NaN,"Марк О’Коннор, Конор МакМахон","Дэйн Уайт О’Хара, Томас Кэйн-Бирн, Джудит Родд...",Семнадцатилетний Дэмиен мечтает вырваться за п...,"Среди, камней, 2019, Россия"


In [7]:
interactions

,user_id,item_id,last_watch_dt,total_dur,watched_pct
0,176549,9506,2021-05-11,4250,72.0
1,699317,1659,2021-05-29,8317,100.0
2,656683,7107,2021-05-09,10,0.0
3,864613,7638,2021-07-05,14483,100.0
4,964868,9506,2021-04-30,6725,100.0
...,...,...,...,...,...
5476246,648596,12225,2021-08-13,76,0.0
5476247,546862,9673,2021-04-13,2308,49.0
5476248,697262,15297,2021-08-20,18307,63.0
5476249,384202,16197,2021-04-19,6203,100.0


## Подготовка данных

In [8]:
Columns.Datetime = 'last_watch_dt'
interactions.drop(interactions[interactions[Columns.Datetime].str.len() != 10].index, inplace=True)
interactions[Columns.Datetime] = pd.to_datetime(interactions[Columns.Datetime], format='%Y-%m-%d')

In [9]:
max_date = interactions[Columns.Datetime].max()

In [10]:
interactions[Columns.Weight] = np.where(interactions['watched_pct'] > 10, 3, 1)

In [11]:
train = interactions[interactions[Columns.Datetime] < max_date - pd.Timedelta(days=7)].copy()
test = interactions[interactions[Columns.Datetime] >= max_date - pd.Timedelta(days=7)].copy()

print(f"train: {train.shape}")
print(f"test: {test.shape}")

train: (4985269, 6)
test: (490982, 6)


In [12]:
train.drop(train.query("total_dur < 300").index, inplace=True)

In [13]:
# отфильтруем холодных пользователей из теста
cold_users = set(test[Columns.User]) - set(train[Columns.User])

In [14]:
test.drop(test[test[Columns.User].isin(cold_users)].index, inplace=True)

## Подготовка фичей

### Пользовательские фичи

In [15]:
# Сколько в % незаполненных полей у Пользователя
users.isnull().sum()/len(users)

user_id     0.000000
age         0.016776
income      0.017586
sex         0.016462
kids_flg    0.000000
dtype: float64

In [16]:
# % не большой => можно заменить пустые значения
users.fillna('Unknown', inplace=True)

In [17]:
# оставляем у df users только тех, кто попал в train
users = users.loc[users[Columns.User].isin(train[Columns.User])].copy()

In [18]:
users

,user_id,age,income,sex,kids_flg
0,973171,age_25_34,income_60_90,М,1
1,962099,age_18_24,income_20_40,М,0
3,721985,age_45_54,income_20_40,Ж,0
4,704055,age_35_44,income_60_90,Ж,0
5,1037719,age_45_54,income_60_90,М,0
...,...,...,...,...,...
840184,529394,age_25_34,income_40_60,Ж,0
840186,80113,age_25_34,income_40_60,Ж,0
840188,312839,age_65_inf,income_60_90,Ж,0
840189,191349,age_45_54,income_40_60,М,1


In [19]:
user_features_frames = []
for feature in ["sex", "age", "income"]:
    feature_frame = users.reindex(columns=[Columns.User, feature])
    feature_frame.columns = ["id", "value"]
    feature_frame["feature"] = feature
    user_features_frames.append(feature_frame)
user_features = pd.concat(user_features_frames)
user_features.head()

,id,value,feature
0,973171,М,sex
1,962099,М,sex
3,721985,Ж,sex
4,704055,Ж,sex
5,1037719,М,sex


### Фичи фильмов

In [20]:
# Сколько в % незаполненных полей у Item
items.isnull().sum()/len(items)

item_id         0.000000
content_type    0.000000
title           0.000000
title_orig      0.297250
release_year    0.006139
genres          0.000000
countries       0.002318
for_kids        0.964543
age_rating      0.000125
studios         0.933283
directors       0.094531
actors          0.164067
description     0.000125
keywords        0.026499
dtype: float64

In [21]:
# оставляем у df users только тех, кто попал в train
items = items.loc[items[Columns.Item].isin(train[Columns.Item])].copy()

### Жанр

In [22]:
items["genre"] = items["genres"].str.lower().str.replace(", ", ",", regex=False).str.split(",")
genre_feature = items[["item_id", "genre"]].explode("genre")
genre_feature.columns = ["id", "value"]
genre_feature["feature"] = "genre"
genre_feature.head()

,id,value,feature
0,10711,драмы,genre
0,10711,зарубежные,genre
0,10711,детективы,genre
0,10711,мелодрамы,genre
1,2508,зарубежные,genre


### Содержание

In [23]:
content_feature = items.reindex(columns=[Columns.Item, "content_type"])
content_feature.columns = ["id", "value"]
content_feature["feature"] = "content_type"
content_feature.head()

,id,value,feature
0,10711,film,content_type
1,2508,film,content_type
2,10716,film,content_type
3,7868,film,content_type
4,16268,film,content_type


### Режисер

In [24]:
items["director"] = items["directors"].str.lower().str.replace(", ", ",", regex=False).str.split(",")
director_feature = items[["item_id", "director"]].explode("director")
director_feature.columns = ["id", "value"]
director_feature["feature"] = "director"
director_feature.head()

,id,value,feature
0,10711,педро альмодовар,director
1,2508,скот армстронг,director
2,10716,адам п. калтраро,director
3,7868,эндрю хэй,director
4,16268,виктор садовский,director


### Страна

In [25]:
items["country"] = items["countries"].str.lower().str.replace(", ", ",", regex=False).str.split(",")
country_feature = items[["item_id", "country"]].explode("country")
country_feature.columns = ["id", "value"]
country_feature["feature"] = "country"
country_feature.head()

,id,value,feature
0,10711,испания,country
1,2508,сша,country
2,10716,канада,country
3,7868,великобритания,country
4,16268,ссср,country


### Год Выпуска

In [26]:
year_feature = items.reindex(columns=[Columns.Item, "release_year"])
year_feature.columns = ["id", "value"]
year_feature["feature"] = "release_year"
year_feature.head()

,id,value,feature
0,10711,2002.0,release_year
1,2508,2014.0,release_year
2,10716,2011.0,release_year
3,7868,2015.0,release_year
4,16268,1978.0,release_year


In [27]:
items

,item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords,genre,director,country
0,10711,film,Поговори с ней,Hable con ella,2002.0,"драмы, зарубежные, детективы, мелодрамы",Испания,NaN,16.0,NaN,Педро Альмодовар,"Адольфо Фернандес, Ана Фернандес, Дарио Гранди...",Мелодрама легендарного Педро Альмодовара «Пого...,"Поговори, ней, 2002, Испания, друзья, любовь, ...","[драмы, зарубежные, детективы, мелодрамы]",[педро альмодовар],[испания]
1,2508,film,Голые перцы,Search Party,2014.0,"зарубежные, приключения, комедии",США,NaN,16.0,NaN,Скот Армстронг,"Адам Палли, Брайан Хаски, Дж.Б. Смув, Джейсон ...",Уморительная современная комедия на популярную...,"Голые, перцы, 2014, США, друзья, свадьбы, прео...","[зарубежные, приключения, комедии]",[скот армстронг],[сша]
2,10716,film,Тактическая сила,Tactical Force,2011.0,"криминал, зарубежные, триллеры, боевики, комедии",Канада,NaN,16.0,NaN,Адам П. Калтраро,"Адриан Холмс, Даррен Шалави, Джерри Вассерман,...",Профессиональный рестлер Стив Остин («Все или ...,"Тактическая, сила, 2011, Канада, бандиты, ганг...","[криминал, зарубежные, триллеры, боевики, коме...",[адам п. калтраро],[канада]
3,7868,film,45 лет,45 Years,2015.0,"драмы, зарубежные, мелодрамы",Великобритания,NaN,16.0,NaN,Эндрю Хэй,"Александра Риддлстон-Барретт, Джеральдин Джейм...","Шарлотта Рэмплинг, Том Кортни, Джеральдин Джей...","45, лет, 2015, Великобритания, брак, жизнь, лю...","[драмы, зарубежные, мелодрамы]",[эндрю хэй],[великобритания]
4,16268,film,Все решает мгновение,NaN,1978.0,"драмы, спорт, советские, мелодрамы",СССР,NaN,12.0,Ленфильм,Виктор Садовский,"Александр Абдулов, Александр Демьяненко, Алекс...",Расчетливая чаровница из советского кинохита «...,"Все, решает, мгновение, 1978, СССР, сильные, ж...","[драмы, спорт, советские, мелодрамы]",[виктор садовский],[ссср]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15958,6443,series,Полярный круг,Arctic Circle,2018.0,"драмы, триллеры, криминал","Финляндия, Германия",NaN,16.0,NaN,Ханну Салонен,"Иина Куустонен, Максимилиан Брюкнер, Пихла Вии...","Во время погони за браконьерами по лесу, сотру...","убийство, вирус, расследование преступления, н...","[драмы, триллеры, криминал]",[ханну салонен],"[финляндия, германия]"
15959,2367,series,Надежда,NaN,2020.0,"драмы, боевики",Россия,0.0,18.0,NaN,Елена Хазанова,"Виктория Исакова, Александр Кузьмин, Алексей М...",Оригинальный киносериал от создателей «Бывших»...,"Надежда, 2020, Россия","[драмы, боевики]",[елена хазанова],[россия]
15960,10632,series,Сговор,Hassel,2017.0,"драмы, триллеры, криминал",Россия,0.0,18.0,NaN,"Эшреф Рейбрук, Амир Камдин, Эрик Эгер","Ола Рапас, Алиетт Офейм, Уильма Лиден, Шанти Р...",Криминальная драма по мотивам романов о шведск...,"Сговор, 2017, Россия","[драмы, триллеры, криминал]","[эшреф рейбрук, амир камдин, эрик эгер]",[россия]
15961,4538,series,Среди камней,Darklands,2019.0,"драмы, спорт, криминал",Россия,0.0,18.0,NaN,"Марк О’Коннор, Конор МакМахон","Дэйн Уайт О’Хара, Томас Кэйн-Бирн, Джудит Родд...",Семнадцатилетний Дэмиен мечтает вырваться за п...,"Среди, камней, 2019, Россия","[драмы, спорт, криминал]","[марк о’коннор, конор макмахон]",[россия]


In [28]:
# Объединяем фичи
item_features = pd.concat((genre_feature, content_feature, country_feature, year_feature, director_feature))
item_features

,id,value,feature
0,10711,драмы,genre
0,10711,зарубежные,genre
0,10711,детективы,genre
0,10711,мелодрамы,genre
1,2508,зарубежные,genre
...,...,...,...
15960,10632,амир камдин,director
15960,10632,эрик эгер,director
15961,4538,марк о’коннор,director
15961,4538,конор макмахон,director


## Метрики

In [29]:
metrics_name = {
    'Precision': Precision,
    'Recall': Recall,
    'MAP': MAP,
}

metrics = {}
for metric_name, metric in metrics_name.items():
    for k in range(1, 11):
        metrics[f'{metric_name}@{k}'] = metric(k=k)


pprint(metrics)

{'MAP@1': MAP(k=1, divide_by_k=False),
 'MAP@10': MAP(k=10, divide_by_k=False),
 'MAP@2': MAP(k=2, divide_by_k=False),
 'MAP@3': MAP(k=3, divide_by_k=False),
 'MAP@4': MAP(k=4, divide_by_k=False),
 'MAP@5': MAP(k=5, divide_by_k=False),
 'MAP@6': MAP(k=6, divide_by_k=False),
 'MAP@7': MAP(k=7, divide_by_k=False),
 'MAP@8': MAP(k=8, divide_by_k=False),
 'MAP@9': MAP(k=9, divide_by_k=False),
 'Precision@1': Precision(k=1),
 'Precision@10': Precision(k=10),
 'Precision@2': Precision(k=2),
 'Precision@3': Precision(k=3),
 'Precision@4': Precision(k=4),
 'Precision@5': Precision(k=5),
 'Precision@6': Precision(k=6),
 'Precision@7': Precision(k=7),
 'Precision@8': Precision(k=8),
 'Precision@9': Precision(k=9),
 'Recall@1': Recall(k=1),
 'Recall@10': Recall(k=10),
 'Recall@2': Recall(k=2),
 'Recall@3': Recall(k=3),
 'Recall@4': Recall(k=4),
 'Recall@5': Recall(k=5),
 'Recall@6': Recall(k=6),
 'Recall@7': Recall(k=7),
 'Recall@8': Recall(k=8),
 'Recall@9': Recall(k=9)}


## Подбор гиперпараметртов

In [ ]:
dataset = Dataset.construct(
    interactions_df=train,
    user_features_df=user_features,
    cat_user_features=["sex", "age", "income"],
    item_features_df=item_features,
    cat_item_features=["genre", "content_type", "director", "country", "release_year"],
)

TEST_USERS = test[Columns.User].unique()

In [32]:
results_opto = []
def objective(trial):
    # общие параметры
    K_RECOS = 10
    RANDOM_STATE = 42
    NUM_THREADS = 4
    N_FACTORS = trial.suggest_int("factors", 32, 64, 32, log=False)

    reco_model = trial.suggest_categorical("reco_model", ["ALS", "LightFM"])
    model_quality_opto = {"model": f"{reco_model}_{trial.number}"}

    if reco_model == "ALS":
        # гиперпараметры для ALS
        regularization = trial.suggest_float("regularization",
                                             1e-4,
                                             1.0,
                                             log=False)
        iterations = trial.suggest_int("iterations", 1, 6, 2, log=False)
        is_fitting_features = True
        # Инициализация ALS
        model = ImplicitALSWrapperModel(
            model=AlternatingLeastSquares(factors=N_FACTORS,
                                 random_state=RANDOM_STATE,
                                 num_threads=NUM_THREADS,
                                 regularization=regularization,
                                 iterations=iterations),
            fit_features_together=is_fitting_features,
        )
    else:
        # гиперпараметры для Lightfm
        N_EPOCHS = trial.suggest_int("epochs", 1, 50, 10, log=False)  # Lightfm
        USER_ALPHA = trial.suggest_float("user_alpha",
                                         1e-3,
                                         1.0,
                                         log=True)  # Lightfm
        ITEM_ALPHA = trial.suggest_float("item_alpha", 1e-3, 1.0,
                                         log=True)  # Lightfm
        LEARNING_RATE = trial.suggest_float("learning_rate",
                                            1e-4,
                                            1.0,
                                            log=True)  # Lightfm
        # Инициализация Lightfm
        model = LightFMWrapperModel(
            LightFM(
                no_components=N_FACTORS,
                loss=loss,
                random_state=RANDOM_STATE,
                learning_rate=LEARNING_RATE,
                user_alpha=USER_ALPHA,
                item_alpha=ITEM_ALPHA,
            ),
            epochs=N_EPOCHS,
            num_threads=NUM_THREADS,
        )

    # обучение модели
    model.fit(dataset)
    recos = model.recommend(
        users=TEST_USERS,
        dataset=dataset,
        k=K_RECOS,
        filter_viewed=True,
    )

    # Подсчет метрик
    metric_values = calc_metrics(metrics, recos, test, train)
    model_quality_opto.update(metric_values)
    results_opto.append(model_quality_opto)

    return metric_values.get('MAP@10') # максимизируемая метрика

In [ ]:
# запуск подбора гиперпараметров
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=1)

pprint(f"Number of finished trials: {len(study.trials)}")
trial = study.best_trial
pprint(f"Best trial: {trial}")

## Approximate Nearest Neighbors

In [30]:
K_RECOS = 10
RANDOM_STATE = 42
NUM_THREADS = 4
N_FACTORS = 64
N_EPOCHS = 1 # Lightfm
USER_ALPHA = 0 # Lightfm
ITEM_ALPHA = 0 # Lightfm
LEARNING_RATE = 0.03 # Lightfm

In [34]:
model = LightFMWrapperModel(
            LightFM(
                no_components=N_FACTORS,
                loss='warp',
                random_state=RANDOM_STATE,
                learning_rate=LEARNING_RATE,
                user_alpha=USER_ALPHA,
                item_alpha=ITEM_ALPHA,
            ),
            epochs=N_EPOCHS,
            num_threads=NUM_THREADS,
        )

model.fit(dataset)

In [35]:
recos = model.recommend(
    users=TEST_USERS,
    dataset=dataset,
    k=K_RECOS,
    filter_viewed=True,
)

In [36]:
results = []
model_quality = {'model': 'LightFM'}
metric_values = calc_metrics(metrics, recos, test, train)
model_quality.update(metric_values)
results.append(model_quality)

df_quality = pd.DataFrame(results).T

df_quality.columns = df_quality.iloc[0]

df_quality.drop('model', inplace=True)

In [37]:
df_quality.style.highlight_max(color='lightgreen', axis=1)

model,LightFM
Precision@1,0.081309
Recall@1,0.041500
Precision@2,0.069231
Recall@2,0.068953
Precision@3,0.061603
Recall@3,0.090576
Precision@4,0.055854
Recall@4,0.108265
Precision@5,0.050591
Recall@5,0.121540


In [39]:
dill_file = Path().cwd().parent / 'service' / 'models'

with open(dill_file / 'LightFM_0.077731.dill', 'wb') as f:
    dill.dump(model, f)

In [40]:
dill_file = Path().cwd().parent / 'service' / 'data'

with open(dill_file / 'dataset_LightFM_0.077731.dill', 'wb') as f:
    dill.dump(dataset, f)

In [38]:
user_embeddings, item_embeddings = model.get_vectors(dataset)

In [41]:
user_embeddings.shape, item_embeddings.shape

((756562, 66), (14019, 66))

In [42]:
def augment_inner_product(factors):
    normed_factors = np.linalg.norm(factors, axis=1)
    max_norm = normed_factors.max()

    extra_dim = np.sqrt(max_norm ** 2 - normed_factors ** 2).reshape(-1, 1)
    augmented_factors = np.append(factors, extra_dim, axis=1)
    return max_norm, augmented_factors

In [43]:
print('pre shape: ', item_embeddings.shape)
max_norm, augmented_item_embeddings = augment_inner_product(item_embeddings)
augmented_item_embeddings.shape

pre shape:  (14019, 66)


(14019, 67)

In [44]:
extra_zero = np.zeros((user_embeddings.shape[0], 1))
augmented_user_embeddings = np.append(user_embeddings, extra_zero, axis=1)
augmented_user_embeddings.shape

(756562, 67)

In [45]:
user_id = 30
user_embeddings[user_id]

array([-1.35195312e+02,  1.00000000e+00, -5.07179871e-02,  1.44656748e-01,
        7.02981800e-02, -1.21144965e-01, -2.19463155e-01,  1.41464576e-01,
       -5.44647053e-02, -6.92157298e-02,  6.17435724e-02,  7.99831226e-02,
        5.30064367e-02,  3.90940070e-01,  7.37713575e-02,  1.59006089e-01,
       -2.63367426e-02,  9.42614079e-02, -2.08240658e-01, -1.92855075e-02,
       -1.12979904e-01,  1.18719414e-01, -1.52341485e-01,  1.23634003e-02,
       -1.71090305e-01, -1.12111188e-01,  1.70729250e-01, -5.50639331e-02,
        1.69594646e-01, -2.19219670e-01,  9.96643677e-02, -9.83265415e-02,
       -7.87180811e-02,  5.50515503e-02, -1.19395040e-01,  2.34548878e-02,
        1.48493022e-01,  2.11184829e-01, -2.09682792e-01, -1.71188921e-01,
        8.72102231e-02,  7.22820088e-02, -2.60722525e-02, -2.36487910e-01,
        6.66145980e-02, -8.33640471e-02,  1.53124034e-01, -6.63255155e-02,
        8.99285916e-03, -6.08504675e-02, -2.47740239e-01, -5.37916496e-02,
        1.33499190e-01, -

In [46]:
augmented_user_embeddings[user_id]

array([-1.35195312e+02,  1.00000000e+00, -5.07179871e-02,  1.44656748e-01,
        7.02981800e-02, -1.21144965e-01, -2.19463155e-01,  1.41464576e-01,
       -5.44647053e-02, -6.92157298e-02,  6.17435724e-02,  7.99831226e-02,
        5.30064367e-02,  3.90940070e-01,  7.37713575e-02,  1.59006089e-01,
       -2.63367426e-02,  9.42614079e-02, -2.08240658e-01, -1.92855075e-02,
       -1.12979904e-01,  1.18719414e-01, -1.52341485e-01,  1.23634003e-02,
       -1.71090305e-01, -1.12111188e-01,  1.70729250e-01, -5.50639331e-02,
        1.69594646e-01, -2.19219670e-01,  9.96643677e-02, -9.83265415e-02,
       -7.87180811e-02,  5.50515503e-02, -1.19395040e-01,  2.34548878e-02,
        1.48493022e-01,  2.11184829e-01, -2.09682792e-01, -1.71188921e-01,
        8.72102231e-02,  7.22820088e-02, -2.60722525e-02, -2.36487910e-01,
        6.66145980e-02, -8.33640471e-02,  1.53124034e-01, -6.63255155e-02,
        8.99285916e-03, -6.08504675e-02, -2.47740239e-01, -5.37916496e-02,
        1.33499190e-01, -

In [47]:
item_id = 0
item_embeddings[item_id]

array([ 1.        ,  2.04707956, -0.54668128,  0.65530181, -0.44489145,
        0.41983646, -0.46595898, -0.24688846, -0.03859854, -0.65956789,
        0.67722702,  0.02954945, -0.03190574,  0.04307969, -0.22766106,
        0.47279146,  0.573421  , -0.34485427, -0.86445564, -0.48088846,
       -0.58310789, -0.1844137 , -0.45517009, -0.91166395, -0.13683847,
       -0.0149786 , -0.35231403, -0.37058502,  0.07523946, -0.1396734 ,
       -0.24115126, -0.23940909, -0.1143521 ,  0.35653773, -0.52283114,
       -0.25274023, -0.19360745,  0.19582717, -0.32942533, -0.49729905,
       -0.37277198,  0.13493618, -0.13943531, -0.5724445 ,  0.17464188,
        0.0078528 ,  0.61789888, -0.64239192,  0.52847207,  0.342978  ,
        0.08393496, -0.03035552, -0.35069817, -0.40612835, -0.02388753,
       -0.51229227,  0.78586411, -0.90114743, -0.54897767,  0.3628284 ,
        0.72209895,  0.29111034,  0.20579137, -0.17773563,  0.46362245,
       -0.12279855])

In [48]:
augmented_item_embeddings[item_id]

array([ 1.        ,  2.04707956, -0.54668128,  0.65530181, -0.44489145,
        0.41983646, -0.46595898, -0.24688846, -0.03859854, -0.65956789,
        0.67722702,  0.02954945, -0.03190574,  0.04307969, -0.22766106,
        0.47279146,  0.573421  , -0.34485427, -0.86445564, -0.48088846,
       -0.58310789, -0.1844137 , -0.45517009, -0.91166395, -0.13683847,
       -0.0149786 , -0.35231403, -0.37058502,  0.07523946, -0.1396734 ,
       -0.24115126, -0.23940909, -0.1143521 ,  0.35653773, -0.52283114,
       -0.25274023, -0.19360745,  0.19582717, -0.32942533, -0.49729905,
       -0.37277198,  0.13493618, -0.13943531, -0.5724445 ,  0.17464188,
        0.0078528 ,  0.61789888, -0.64239192,  0.52847207,  0.342978  ,
        0.08393496, -0.03035552, -0.35069817, -0.40612835, -0.02388753,
       -0.51229227,  0.78586411, -0.90114743, -0.54897767,  0.3628284 ,
        0.72209895,  0.29111034,  0.20579137, -0.17773563,  0.46362245,
       -0.12279855,  6.64016828])

In [49]:
# Set index parameters
# These are the most important ones
M = 48
efC = 100

# Number of neighbors
K=10

# Space name should correspond to the space name
# used for brute-force search
space_name='negdotprod'

num_threads = 4
index_time_params = {'M': M, 'indexThreadQty': num_threads, 'efConstruction': efC, 'post' : 0}
print('Index-time parameters', index_time_params)

Index-time parameters {'M': 48, 'indexThreadQty': 4, 'efConstruction': 100, 'post': 0}


In [50]:

# Intitialize the library, specify the space, the type of the vector and add data points
index = nmslib.init(method='hnsw', space=space_name, data_type=nmslib.DataType.DENSE_VECTOR)
index.addDataPointBatch(augmented_item_embeddings)

14019

In [51]:
# Create an index
start = time.time()
index_time_params = {'M': M, 'indexThreadQty': num_threads, 'efConstruction': efC}
index.createIndex(index_time_params)
end = time.time()
print('Index-time parameters', index_time_params)
print('Indexing time = %f' % (end-start))

Index-time parameters {'M': 48, 'indexThreadQty': 4, 'efConstruction': 100}
Indexing time = 0.754718


In [52]:
# Setting query-time parameters
efS = 100
query_time_params = {'efSearch': efS}
print('Setting query-time parameters', query_time_params)
index.setQueryTimeParams(query_time_params)

Setting query-time parameters {'efSearch': 100}


In [53]:
user_index = np.where(dataset.user_id_map.external_ids == 1032142)[0][0]
query_matrix = augmented_user_embeddings[user_index]

In [55]:
query_matrix

array([-0.32985109,  1.        ,  0.02205707,  0.08980627,  0.20087105,
       -0.15627137, -0.14015584,  0.18227796,  0.01683466, -0.06439552,
        0.15855706, -0.02145658,  0.04277911,  0.10165393,  0.04622089,
        0.09623418,  0.0345323 ,  0.13958901, -0.09586219,  0.19127336,
        0.12665424,  0.10065325, -0.15001273,  0.14071476, -0.08976087,
        0.13960318,  0.01710628,  0.21610653,  0.02155385,  0.18535024,
        0.14548849, -0.09528606, -0.13279074,  0.16046426, -0.07223549,
        0.06873992,  0.03054859,  0.09696586, -0.15979564, -0.1096307 ,
        0.08536576,  0.11428012,  0.12382121, -0.05757909,  0.07101226,
        0.07844304,  0.10848665,  0.08020473,  0.0803762 , -0.10346356,
       -0.03905372, -0.24272341,  0.01137958,  0.0152712 , -0.05923871,
       -0.11694247,  0.02307201,  0.03915438, -0.12259786,  0.06217144,
       -0.063027  , -0.13929345,  0.09878232, -0.16415736, -0.00595621,
        0.09892885,  0.        ])

In [56]:
# Querying
query_qty = query_matrix.shape[0]
start = time.time()
nbrs = index.knnQueryBatch([query_matrix], k = K, num_threads = num_threads)
end = time.time()
print('kNN time total=%f (sec), per query=%f (sec), per query adjusted for thread number=%f (sec)' %
      (end-start, float(end-start)/query_qty, num_threads*float(end-start)/query_qty))

kNN time total=0.006853 (sec), per query=0.000102 (sec), per query adjusted for thread number=0.000409 (sec)


In [57]:
nbrs = index.knnQueryBatch([query_matrix], k = K, num_threads = num_threads)

In [59]:
nbrs

[(array([ 19,  43,  31,  32, 121,  69, 173,  36,  62, 100], dtype=int32),
  array([-4.6914086, -4.545101 , -4.415558 , -4.3382797, -4.122013 ,
         -4.0413547, -4.0412517, -4.038016 , -3.9738822, -3.9712205],
        dtype=float32))]

In [60]:
dataset.user_id_map.external_ids[nbrs[0][0]]

array([ 916352,  496279,  513902,  568405,  907474, 1058641,  393684,
        727498,  120361,  868380])

In [65]:
user_id = 6064
K_RECOS = 10
N = 50
cnt = Counter(train['item_id'].values)
idf = pd.DataFrame.from_dict(cnt, orient='index', columns=['doc_freq']).reset_index()
n = train.shape[0]
idf['idf'] = idf['doc_freq'].apply(lambda x: np.log((1 + n) / (1 + x) + 1))

users_inv_mapping = dict(enumerate(train['user_id'].unique()))
users_mapping = {v: k for k, v in users_inv_mapping.items()}


items_inv_mapping = dict(enumerate(train['item_id'].unique()))
items_mapping = {v: k for k, v in items_inv_mapping.items()}


def generate_implicit_recs_mapper(model, N, users_mapping, users_inv_mapping):
    def _recs_mapper(user):
        user_id = users_mapping[user]
        recs = model.similar_items(user_id, N=N)
        return [users_inv_mapping[user] for user, _ in recs], [sim for _, sim in recs]
    return _recs_mapper

In [64]:
mapper = generate_implicit_recs_mapper(
    userknn,
    N=N,
    users_mapping=users_mapping,
    users_inv_mapping=users_inv_mapping
)

NameError: name 'userknn' is not defined

In [71]:
watched = train.groupby('user_id').agg({'item_id': list})
watched.head()

,item_id
user_id,
0,[15297]
1,[10440]
2,"[7571, 3541, 15266, 13867, 12841, 10770, 4475,..."
3,"[12192, 9728, 16406, 15719, 10440, 3475, 4151,..."
4,[4700]


In [72]:
wwatched = {k:v['item_id'] for k,v in json.loads(watched.T.to_json()).items()}

In [122]:
# user_id = 33
def recommend_nmslib(user_id, k_recos):
    recss = {}
    user_index = np.where(dataset.user_id_map.external_ids == user_id)
    if len(user_index[0]):
        user_index = user_index[0][0]
        query_matrix = augmented_user_embeddings[user_index]
        nbrs = index.knnQueryBatch([query_matrix],
                                   k=K,
                                   num_threads=num_threads)
        recss['similar_user_id'], recss[
            'similarity'] = dataset.user_id_map.external_ids[nbrs[0][0]], nbrs[0][1]
        recss['item_id'] = [
            wwatched.get(f"{x}") for x in recss['similar_user_id']
        ]
        recs = pd.DataFrame(recss)
        recs = recs.explode('item_id')
        recs = recs.sort_values(['similarity'], ascending=False)
        recs = recs.merge(idf[['index', 'idf']],
                          left_on='item_id',
                          right_on='index',
                          how='left').drop(['index'], axis=1)
        recs['rank_idf'] = recs['similarity'] * recs['idf']
        recs = recs.sort_values(['rank_idf'], ascending=False)
        recs.dropna(inplace=True)
        return recs['item_id'].unique()[:k_recos]
    else:
        return None

In [120]:
recss = {}
user_index = np.where(dataset.user_id_map.external_ids == user_id)
if len(user_index[0]):
    user_index = user_index[0][0]
    query_matrix = augmented_user_embeddings[user_index]
    nbrs = index.knnQueryBatch([query_matrix],
                               k=K,
                               num_threads=num_threads)
    recss['similar_user_id'], recss[
        'similarity'] = dataset.user_id_map.external_ids[nbrs[0][0]], nbrs[0][1]
#     recss['similar_user_id'] = recss['similar_user_id'][1:]

In [121]:
recss

{'similar_user_id': array([1093823,  727498,  713676,  791466,  273214, 1058641,   82721,
         532906,  179618,  827691]),
 'similarity': array([117.45208 , 117.51571 , 117.551575, 117.59176 , 117.738365,
        117.745056, 117.77718 , 117.77753 , 117.80196 , 117.817215],
       dtype=float32)}

In [58]:
# получение рекомендации по конкретному пользователю
def recommend_nmslib(user_id, K=10, num_threads=4):
    user_index = np.where(dataset.user_id_map.external_ids == user_id)[0][0]
    query_matrix = augmented_user_embeddings[user_index]
    nbrs = index.knnQueryBatch([query_matrix], k = K, num_threads = num_threads)
    labels = dataset.item_id_map.external_ids[nbrs[0][0]]
    distances = nbrs[0][1]
    return labels, distances

In [123]:
res = []
for user in TEST_USERS[:]:
    user_index = np.where(dataset.user_id_map.external_ids == user)
    if len(user_index[0]):
        labels = recommend_nmslib(user, k_recos=10)
        labels_u = np.zeros((len(labels), 3), dtype=int)
        labels_u[:, 0] = user

        # проставляем порядок треков
        labels_u[:, 2] = range(len(labels))
        labels_u[:, 1] = labels
    #     labels_u[:, 2] = distances
        res.extend(labels_u)

In [124]:
recos_ = pd.DataFrame(res, columns=['user_id', 'item_id', 'rank'])

In [125]:
# подсчет метрик
results = []
model_quality = {'model': 'nmslib'}
metric_values = calc_metrics(metrics, recos_, test, train)
model_quality.update(metric_values)
results.append(model_quality)

df_quality = pd.DataFrame(results).T

df_quality.columns = df_quality.iloc[0]

df_quality.drop('model', inplace=True)

In [126]:
df_quality.style.highlight_max(color='lightgreen', axis=1)

model,nmslib
Precision@1,0.022420
Recall@1,0.011912
Precision@2,0.015282
Recall@2,0.015707
Precision@3,0.012628
Recall@3,0.018821
Precision@4,0.011066
Recall@4,0.021822
Precision@5,0.010190
Recall@5,0.024857


In [59]:
def recommend_all(query_factors, index_factors, topn=10):
    output = query_factors.dot(index_factors.T)
    argpartition_indices = np.argpartition(output, -topn)[:, -topn:]

    x_indices = np.repeat(np.arange(output.shape[0]), topn)
    y_indices = argpartition_indices.flatten()
    top_value = output[x_indices, y_indices].reshape(output.shape[0], topn)
    top_indices = np.argsort(top_value)[:, ::-1]

    y_indices = top_indices.flatten()
    top_indices = argpartition_indices[x_indices, y_indices]
    labels = top_indices.reshape(-1, topn)
    distances = output[x_indices, top_indices].reshape(-1, topn)
    return labels, distances
